## Open Government Data, provided by **OpenDataZurich**
*Autogenerated Python starter code for data set with identifier* **geo_stillgewaesserinventar**

## Dataset
# **Stillgewaesserinventar**

## Description

Inventar aller bekannten Weiher, Wasserbecken, Geschiebefänge und Staubereiche, wo mit Amphibien- und/oder Libellenvorkommen gerechnet wird. Die Liste ist im GSZ sowie ERZ Zuständigkeitsbereich abschliessend, nicht aber im restlichen Siedlungsgebiet (insbesondere Privatflächen).

**Zweck**: Stillgewässer sind aufgrund ihrer hohen Biodiversität von grosser Bedeutung für den Naturschutz. Besonders wertvoll sind Gewässer mit den Leitarten Amphibien und/oder Libellen. Solche Stillgewässer wirken meist auch anziehend auf Erholungssuchende. Stillgewässerverluste sollen minimiert und wenn immer möglich kompensiert werden. Für deren Erhalt müssen Stillgewässer regelmässig gepflegt und von Zeit zu Zeit saniert werden. 
Der Datensatz dient als Planungsgrundlage für verschiedene Tätigkeiten: Bestimmung und Priorisierung der Pflegeeingriffe, Förderung des Biotopverbunds, Steuerung der Erholungsnutzung, Beurteilung von Baugesuchen und -Projekten (Baukoordination). Inwiefern und in welchem Umfang Massnahmen erforderlich sind, wird im Einzelfall geprüft.

**Genereller Hinweis zum Geodatensatz:**


Weitere Informationen zu diesem Datensatz finden sie unter [«**Stillgewaesserinventar»** auf Geocat.ch](https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/68a6d8f1-851c-4d6f-b490-7efb9eb3da88).				
			   

**Informationen zum Datensatz:**

Verwendung in der Baukoordination

**Datenerfassung:**

Die Polygone der Stillgewässer werden seit 2015 von einer Person zusammengetragen. Sie sind ein Sammelsurium aus der Bodenbedeckung (BB), Biotoptypenkartierung (BTK), Faunakartierung (FK), Flächenmanagement (FMS), Kanalinformationssystem (KIS) oder Waldentwicklungsplan (WEP). Teilweise sind sie händisch basierend auf Orthofotos in GIS eingetragen worden. Es besteht der Anspruch, dass jedes Objekt überprüft wird auf Vorkommen der Ziel- und Leitarten. Die Vorkommen werden im Rahmen der Faunakartierung systematisch gesamtstädtisch erhoben. Die Erhebungen dauern 10 Jahre.

**Datengrundlage:**

Dieser Datenbestand ist von folgenden Datenbeständen abhängig:
-ESRI Shapefiles 
-Uebersichtsplan Stadt Zürich
-Faunakartierung Amphibien und Libellen

**Statisches Vorschaubild:**

![BildText](https://www.gis.stadt-zuerich.ch/zueriplan_docs/geocat/68a6d8f1-851c-4d6f-b490-7efb9eb3da88.png)



## Data set links

[Direct link by OpenDataZurich for dataset](https://data.stadt-zuerich.ch/dataset/geo_stillgewaesserinventar)

https://www.stadt-zuerich.ch/geodaten/download/Stillgewaesserinventar?format=geojson_link<br>


## Metadata
- **Publisher** `Artenförderung und Schutzgebiete, Grün Stadt Zürich, Tiefbau- und Entsorgungsdepartement`
- **Maintainer** `Open Data Zürich`
- **Maintainer_email** `opendata@zuerich.ch`
- **Keywords** `Bevölk­erung,Umwelt,Ver­walt­ung`
- **Tags** `['amphibien', 'artenschutz', 'bauberatung', 'baukoordination', 'biodiversitat', 'biotop', 'fauna', 'fliessgewasser', 'geodaten', 'geoportal', 'geschiebefang', 'geschiebesammler', 'gewasser', 'libellen', 'natur', 'polygondaten', 'schutz', 'stzh', 'teich', 'vektordaten', 'wasserbecken']`
- **Metadata_created** `2023-11-06T03:22:39.234421`
- **Metadata_modified** `2025-12-20T06:38:24.729921`


## Imports and helper functions

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 

import json
import xml.etree.ElementTree as ET
import requests
import geopandas as gpd
import re

In [ ]:
# helper function for reading datasets with proper separator
def get_dataset(url):
    # get URL ofs WFS service
    dataset_identifier = re.search(r'\/([^\/\?]+)\?', url).group(1)
    url_geoportal = f"https://www.ogd.stadt-zuerich.ch/wfs/geoportal/{dataset_identifier}"
    print("Getting available layers from:", url_geoportal)
    
    # Parameter for GetCapabilities
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetCapabilities"
    }
    
    # send GetCapabilities
    response = requests.get(url_geoportal, params=params)

    # parse XML answer
    root = ET.fromstring(response.content)

    # define Namespace 
    namespace = {'wfs': 'http://www.opengis.net/wfs'}
    
    # Exctract available layers
    layers = [feature_type.find('wfs:Name', namespace).text for feature_type in root.findall('.//wfs:FeatureType', namespace)]
    
    print("Available layers:", layers)
    print("First layer is set as default. To chose another layer set it as typename in the get_dataset() function.")

    # set first layer als typename
    typename = layers[0]
    print("Chosen typename:", typename)

    # Parameter GetFeature request
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetFeature",
        "typename": typename,
        "outputFormat": "application/json"
    }
    
    # GetFeature request
    response = requests.get(url_geoportal, params=params)

    # Load GeoJSON in GeoDataFrame
    gdf = gpd.read_file(json.dumps(response.json()))
    return gdf

## Load the data

In [ ]:
gdf = get_dataset('https://www.stadt-zuerich.ch/geodaten/download/Stillgewaesserinventar?format=geojson_link')

## Analyze the data

In [ ]:
gdf.plot()

In [ ]:
# drop columns that have no values
gdf.dropna(how='all', axis=1, inplace=True)

In [ ]:
print(f'The dataset has {gdf.shape[0]:,.0f} rows (observations) and {gdf.shape[1]:,.0f} columns (variables).')
print(f'There seem to be {gdf.duplicated().sum()} exact duplicates in the data.')

In [ ]:
gdf.info(memory_usage='deep', verbose=True)

In [ ]:
gdf.head()

In [ ]:
# display a small random sample transposed in order to see all variables
gdf.sample(3).T

In [ ]:
# describe non-numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(exclude='number'))
except:
    print("No categorical data in dataset.")

In [ ]:
# describe numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(include='number'))
except:
    print("No numercial data in dataset.")

In [ ]:
# plot a histogram for each numerical feature
try:
    gdf.hist(bins=25, rwidth=.9)
    plt.tight_layout()
    plt.show()
except:
    print("No numercial data to plot.") 

In [ ]:
# continue your code here...

**Contact**: opendata@zuerich.ch